In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import *

# Simplified Bootstrap Data

In [2]:
QUOTE_DATE = '2022-12-30'
NDATA = 11

In [3]:
filepath_rawdata = f'../data/treasury_quotes_{QUOTE_DATE}.xlsx'
rawdata = pd.read_excel(filepath_rawdata,sheet_name='quotes')
rawdata.columns = rawdata.columns.str.upper()
rawdata.sort_values('TMATDT',inplace=True)
rawdata.set_index('KYTREASNO',inplace=True)

t_check = rawdata['CALDT'].values[0]
if rawdata['CALDT'].eq(t_check).all():
    t_current = t_check
else:
    warnings.warn('Quotes are from multiple dates.')
    t_current = None

f'Analysis based on quotes from: {pd.to_datetime(t_current):%Y-%m-%d}'

'Analysis based on quotes from: 2022-12-30'

In [4]:
metrics = rawdata.copy()[['TDATDT','TMATDT','TCOUPRT','TDYLD','TDDURATN','TDPUBOUT']]
metrics.columns = ['issue date','maturity date','coupon rate','TDYLD','duration','outstanding']
metrics['TDYLD'] *= 365.25
metrics['duration'] /= 365.25
metrics['outstanding'] *= 1e6
metrics['maturity'] = get_maturity_delta(metrics['maturity date'], t_current)

### Careful

If the quote date is just before a coupon date, might approximate it has already happened when building the cashflow matrix.

For example, quote of 12-30-2022, but coupon is technically 12-31-2022.

This creates problems when using the dirty accrued interest but not modeling that cashflow the next day. 

Accordingly, if quote date is immediately before a coupon, just use the clean price.

In [5]:
USE_DIRTY = False

In [6]:
metrics['price'] = (rawdata['TDBID'] + rawdata['TDASK'])/2
if USE_DIRTY:
    metrics['price'] += rawdata['TDACCINT']
    
metrics['ask-bid'] = rawdata['TDASK'] - rawdata['TDBID']
metrics['next cashflow date'] = (calc_cashflows(rawdata) != 0).idxmax(1)

In [7]:
tab = metrics[['issue date','maturity date','maturity','coupon rate','price','TDYLD']].rename(columns={'TDYLD':'ytm'}).dropna()

idxset = []
for mat in range(1,NDATA):
    idxset.append(tab['maturity'].sub(mat/2).abs().idxmin())

selected_quotes = tab.loc[idxset]

# Calculate Simplified Spot Rates

Get quotes from Fama-Bliss discounts

In [8]:
DATE = '2022-12-30'

filepath = f'../data/treasury_zero_coupon_{DATE}.xlsx'
spot_discounts = pd.read_excel(filepath, sheet_name='prices')
spot_discounts.set_index('date',inplace=True)
spot_discounts.columns = spot_discounts.columns.astype(int)
spot_discounts = spot_discounts.iloc[[-1],:]

Linearly interpolate for semi-annual spacing

In [9]:
for i, mat in enumerate([0.5,1.5,2.5,3.5,4.5]):
    if i==0:
        spot_discounts[mat] = (1 + spot_discounts.iloc[0,i])/2
    else:
        spot_discounts[mat] = (spot_discounts.iloc[0,i-1] + spot_discounts.iloc[0,i])/2
        
sort_columns = np.arange(.5,5.5,.5)
spot_discounts = spot_discounts[sort_columns].T
spot_discounts.columns = ['discount factor']
spot_discounts.index.name = 'maturity'

# Save Data

In [10]:
outfile = f'../data/selected_treasury_quotes_{QUOTE_DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    selected_quotes.to_excel(writer, sheet_name= 'quotes', index=True)

In [11]:
outfile = f'../data/simplified_spot_discounts_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    spot_discounts.to_excel(writer, sheet_name= 'data', index=True)